## Model exploration of a genetic oscillator
Obs. You need to have the StochKit solver installed for this example

In [ ]:
%matplotlib notebook

import gillespy2
from gillespy2.solvers.stochkit import StochKitSolver
from tsfresh.feature_extraction.settings import MinimalFCParameters
import numpy as np
import os

from dask.distributed import Client
#os.environ["STOCHKIT_HOME"] = '/path/to/StochKit'

In [ ]:
model_doc = gillespy2.StochMLDocument.from_file("vilar_oscillator.xml")
vilar_model = model_doc.to_model("Vilar")
vilar_model.tspan = np.linspace(0, 100, 100)

## Define simulator function

In [ ]:
from sciope.utilities.gillespy2 import wrapper

settings = {"solver": StochKitSolver, "number_of_trajectories":1, "show_labels":True}
simulator = wrapper.get_simulator(gillespy_model=vilar_model, run_settings=settings, species_of_interest=["C", "R"])

#get the expression values of parameter as array (used for sampler)
expression_array = wrapper.get_parameter_expression_array(vilar_model)

## Define parameter sampler

In [ ]:
from sciope.utilities.priors.uniform_prior import UniformPrior

sampler = UniformPrior(expression_array*0.8, expression_array*1.5)

## Start local cluster using dask client

In [ ]:
c = Client()
c

## Initiate StochMET

In [ ]:
from sciope.utilities.summarystats.auto_tsfresh import SummariesTSFRESH
from sciope.stochmet.stochmet import StochMET


#lets use this set of features:
default_fc_params = {'mean': None,
                     'variance': None,
                     'skewness': None,
     'agg_autocorrelation': [{'f_agg': 'mean', 'maxlag': 5},
 {'f_agg': 'median', 'maxlag': 5},
 {'f_agg': 'var', 'maxlag': 5}]}

summaries = SummariesTSFRESH(features=default_fc_params)
met = StochMET(simulator, sampler, summarystats=summaries)

## Run parameter sweep

In [ ]:
met.compute(n_points=100, chunk_size=10)

## Explore the result
Here we will explore parameter points expressed in feature space using a dimension reduction method. User can interact with points and label points according to different model behavior

In [ ]:
#First lets add some appropiate information about the model and features for interative purposes
met.data.configurations['listOfParameters'] = list(vilar_model.listOfParameters.keys())
met.data.configurations['listOfSpecies'] = list(vilar_model.listOfSpecies.keys())
met.data.configurations['listOfSummaries'] = met.summaries.features
met.data.configurations['timepoints'] = vilar_model.tspan

In [ ]:
# Here we use UMAP for dimension reduction and collect the data from persited storage
met.explore(dr_method='umap', from_distributed=True)

## Once a few points has been added we can use Semi-supervised learning

In [ ]:
from sciope.models.label_propagation import LPModel
#here lets use the dimension reduction embedding as input data
data = met.dr_model.embedding_

model_lp = LPModel()
#train using basinhopping
model_lp.train(data, met.data.user_labels, min_=0.01, max_=10, niter=50)

In [ ]:
# just to vislualize the result we will map the label distribution to the user_labels (will enable us to see the LP model 
# output when using "explore")

user_labels = np.copy(met.data.user_labels)
met.data.user_labels = model_lp.model.label_distributions_[:,0]

In [ ]:
met.explore(dr_method='umap', from_distributed=False)

In [ ]:
met.data.user_labels = user_labels